## Library import

In [1]:
import numpy as np
import pandas as pd
import random
import numpy as np
from Bio import SeqIO
import matplotlib.pyplot as plt
from operator import itemgetter 
from matplotlib import pyplot
from collections import Counter
import tqdm
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

SMALL_SIZE = 8
MEDIUM_SIZE = 20
BIGGER_SIZE = 20

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


## Data load (IDO1 Expression)

In [ ]:
sc_path= 'patent_experiments/'
baseline_path = 'sfold/'
target_DB = pd.read_csv(sc_path+'aso_features_patent.csv') # Experiments and our features ('gibbs','off_gibbs','secondary_structure')

df_sfold = pd.read_csv(baseline_path+'sfold.csv')
df_sfold_14 = pd.read_excel(baseline_path+'sfold_14mer.xlsx',engine = 'openpyxl')
df_sfold_15 = pd.read_excel(baseline_path+'sfold_15mer.xlsx',engine = 'openpyxl')
df_sfold_16 = pd.read_excel(baseline_path+'sfold_16mer.xlsx',engine = 'openpyxl')
df_sfold_17 = pd.read_excel(baseline_path+'sfold_17mer.xlsx',engine = 'openpyxl') # sfold predictions

## Entire (Linear Regression)

In [ ]:
elements=['gibbs','off_gibbs','secondary_structure'] 

X = target_DB[elements]
Y = target_DB['Inhibition rate'].values
mask = ~target_DB['Inhibition rate'].isna()

X = X[mask]

Y = Y[mask]
Y = 1-Y

X_train=X
y_train=Y
X_test=X
y_test=Y

In [ ]:

reg = LinearRegression(fit_intercept=True).fit(X_train, y_train)
y_pred = reg.predict(X_test)
y_train_pred = reg.predict(X_train)
coef = reg.coef_/reg.coef_[0]

r_squared = r2_score(y_test, y_pred)

fig = plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred,marker = 'D', c='b',s=20)
plt.plot(np.unique(y_test), np.poly1d(np.polyfit(y_test, y_pred, 1))(np.unique(y_test)),color='k')


plt.xlabel('Experimental inhibition rate (%)')
plt.ylabel('Predicted inhibition rate (%)')
plt.xlim([0.1, 1.01])
plt.ylim([0.3,1.01])


print('Correlation: {:.4f}'.format(np.sqrt(r_squared)))
print('R2: {:.4f}'.format(r_squared))

plt.show()

### Statistical analysis

In [ ]:
X_train_copy = X_train.copy()
X_train_copy['y'] = y_train

results = smf.ols('y ~ gibbs + off_gibbs + secondary_structure', data=X_train_copy).fit()

# # Inspect the results
print(results.summary())
print(results.pvalues)

## Entire (Linear Regression, longer samples) 

In [ ]:
target_DB_longer = target_DB[target_DB['len_seq']>16]
elements=['gibbs','off_gibbs','secondary_structure'] 

X = target_DB_longer[elements]
Y = target_DB_longer['Inhibition rate'].values

mask = ~target_DB_longer['Inhibition rate'].isna()


X = X[mask]
Y = Y[mask]
Y = 1-Y

X_train=X
y_train=Y
X_test=X
y_test=Y

In [ ]:
reg = LinearRegression(fit_intercept=True).fit(X_train, y_train)
y_pred = reg.predict(X_test)
y_train_pred = reg.predict(X_train)
coef = reg.coef_/reg.coef_[0]

r_squared = r2_score(y_test, y_pred)

fig = plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred,marker = 'D', c='b',s=20)
plt.plot(np.unique(y_test), np.poly1d(np.polyfit(y_test, y_pred, 1))(np.unique(y_test)),color='k')


plt.xlabel('Experimental inhibition rate (%)')
plt.ylabel('Predicted inhibition rate (%)')
plt.xlim([0.0, 1.01])
plt.ylim([0.3,1.01])


print('Correlation: {:.4f}'.format(np.sqrt(r_squared)))
print('R2: {:.4f}'.format(r_squared))

plt.show()

## sfold-comparison

In [ ]:
sfold_gibbs=[]
for ii in range(len(target_DB)):
    atgc_len=target_DB.iloc[ii]['len_seq']
    if atgc_len==14:
        sfold_gibbs.append(df_sfold_14[df_sfold_14['antisense']==target_DB.iloc[ii]['Sequence']]['gibbs'].values[0])
    elif atgc_len==15:
        sfold_gibbs.append(df_sfold_15[df_sfold_15['antisense']==target_DB.iloc[ii]['Sequence']]['gibbs '].values[0])        
    elif atgc_len==16:
        sfold_gibbs.append(df_sfold_16[df_sfold_16['antisense']==target_DB.iloc[ii]['Sequence']]['gibbs'].values[0])        
    elif atgc_len==17:
        try:        
            sfold_gibbs.append(df_sfold_17[df_sfold_17['antisense']==target_DB.iloc[ii]['Sequence']]['gibbs'].values[0])
        except IndexError:
            sfold_gibbs.append('nan')
    else:
        try:
            sfold_gibbs.append(df_sfold[df_sfold['antisense']==target_DB.iloc[ii]['Sequence']]['gibbs'].values[0])
        except IndexError:
            sfold_gibbs.append('nan')
            
sfold_gibbs = np.array(sfold_gibbs)

# target_DB['sfold'] = sfolds
target_DB['sfold'] = sfold_gibbs
target_DB = target_DB[target_DB['binding_site']!=-1]

In [ ]:
unique_seq=target_DB['Sequence'].unique()
random.seed('2023-08-18')
random.shuffle(unique_seq)

### Train/Test split

In [ ]:
train_seq = unique_seq[:int(0.80*len(unique_seq))-1]
test_seq = unique_seq[int(0.80*len(unique_seq))-1:]


target_DB_train=target_DB[target_DB['Sequence'].isin(train_seq)]
target_DB_test=target_DB[target_DB['Sequence'].isin(test_seq)]
# target_DB_test[target_DB_test['sfold']!=-1]
elements=['gibbs','off_gibbs','secondary_structure'] 

elements2=['sfold']

In [ ]:
X_train = target_DB_train[elements]
X_train_sfold = target_DB_train[elements2]
y_train = target_DB_train['Inhibition rate'].values

mask = ~target_DB_train['Inhibition rate'].isna()

X_train = X_train[mask]
X_train_sfold = X_train_sfold[mask]
y_train = y_train[mask]
y_train = 1-y_train

X_test = target_DB_test[elements]
X_test_sfold = target_DB_test[elements2]
y_test = target_DB_test['Inhibition rate'].values

mask = ~target_DB_test['Inhibition rate'].isna()

X_test = X_test[mask]
X_test_sfold = X_test_sfold[mask]
y_test = y_test[mask]
y_test = 1-y_test

In [ ]:
reg = LinearRegression(fit_intercept=True).fit(X_train, y_train)
y_pred = reg.predict(X_test)
y_train_pred = reg.predict(X_train)
coef = reg.coef_/reg.coef_[0]

r_squared = r2_score(y_test, y_pred)


reg2= LinearRegression(fit_intercept=True).fit(X_train_sfold, y_train)
y_pred_sfold = reg2.predict(X_test_sfold)
y_train_pred_sfold = reg2.predict(X_train_sfold)
coef2 = reg2.coef_/reg2.coef_[0]

r_squared_sfold = r2_score(y_test, y_pred_sfold)


fig = plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred,marker = 'D', c='b',s=20)
plt.scatter(y_test, y_pred_sfold, marker = 's', c='r',s=20)

plt.plot(np.unique(y_test), np.poly1d(np.polyfit(y_test, y_pred, 1))(np.unique(y_test)),color='b')
plt.plot(np.unique(y_test), np.poly1d(np.polyfit(y_test, y_pred_sfold, 1))(np.unique(y_test)),color='r')


plt.xlabel('Experimental inhibition rate (%)')
plt.ylabel('Predicted inhibition rate (%)')
plt.xlim([0.1, 1.01])
plt.ylim([0.48,1.01])


print('Correlation of ASOptimizer: {:.4f}'.format(np.sqrt(r_squared)))
print('R2 of ASOptimizer: {:.4f}'.format(r_squared))

print('Correlation of sfold: {:.4f}'.format(np.sqrt(r_squared_sfold)))
print('R2 of sfold: {:.4f}'.format(r_squared_sfold))

plt.show()